In [136]:
import pandas as pd
import numpy as np
import pytz
from datetime import datetime, timezone, timedelta, date
import datetime as dt
local_tz = pytz.timezone('America/Bogota')
import math
from docplex.mp.model import Model
import json

# Forecast

In [137]:
forecast = pd.read_csv('Forecast.csv')

In [138]:
forecast = forecast.rename(columns={'Forecast 2023 3': '2023-03-01',
                                    'Forecast 2023 4': '2023-04-01',
                                    'Forecast 2023 5': '2023-05-01',
                                    'Forecast 2023 6': '2023-06-01', 
                                    'Forecast 2023 7': '2023-07-01',
                                    'Forecast 2023 8': '2023-08-01',
                                    'Forecast 2023 9': '2023-09-01',
                                    'Forecast 2023 10': '2023-10-01',
                                    'Forecast 2023 11': '2023-11-01',
                                    'Forecast 2023 12': '2023-12-01',
                                    'Forecast 2024 1': '2024-01-01',
                                    'Forecast 2024 2': '2024-02-01',
                                    'Forecast 2024 3': '2024-03-01',
                                    'Forecast 2024 4': '2024-04-01',
                                    'Forecast 2024 5': '2024-05-01',
                                    'Forecast 2024 6': '2024-06-01',
                                    'Forecast 2024 7': '2024-07-01',
                                    'Forecast 2024 8': '2024-08-01',
                                    'Forecast 2024 9': '2024-09-01',
                                    'Forecast 2024 10': '2024-10-01',
                                    'Forecast 2024 11': '2024-11-01',
                                    'Forecast 2024 12': '2024-12-01',
                                   })

In [139]:
forecast = forecast.melt(id_vars=['EAN_TEXT'], 
                         value_vars=['2023-03-01', '2023-04-01', '2023-05-01', 
                                     '2023-06-01', '2023-07-01', '2023-08-01',
                                     '2023-09-01', '2023-10-01', '2023-11-01', 
                                     '2023-12-01', '2024-01-01', '2024-02-01', 
                                     '2024-03-01', '2024-04-01', '2024-05-01',
                                     '2024-06-01', '2024-07-01', '2024-08-01', 
                                     '2024-09-01', '2024-10-01', '2024-11-01', 
                                     '2024-12-01'], var_name='First_Day_Month', value_name='Monthly_Forecast')
forecast.First_Day_Month = pd.to_datetime(forecast.First_Day_Month)
forecast.Monthly_Forecast = forecast.Monthly_Forecast.replace('',0).astype(float)
forecast['Days_Month'] = forecast['First_Day_Month'].apply(lambda date: date.days_in_month)
forecast['Daily_Forecast'] = forecast['Monthly_Forecast']/forecast['Days_Month']
forecast['Month'] = pd.DatetimeIndex(forecast['First_Day_Month']).month
forecast['Year'] = pd.DatetimeIndex(forecast['First_Day_Month']).year

In [140]:
df_forecast = pd.DataFrame(index=pd.date_range(start=dt.date.today(), periods=730, freq='D')).reset_index().rename(columns={'index':'Date'})
df_forecast= df_forecast.merge(pd.DataFrame(forecast['EAN_TEXT'].unique()), how='cross').rename(columns={0:'EAN_TEXT'})
df_forecast['Month'] = pd.DatetimeIndex(df_forecast['Date']).month
df_forecast['Year'] = pd.DatetimeIndex(df_forecast['Date']).year
df_forecast = df_forecast.merge(forecast, how='left', left_on=['EAN_TEXT', 'Month', 'Year'], right_on=['EAN_TEXT', 'Month', 'Year'])
df_forecast = df_forecast.drop(columns=['First_Day_Month', 'Days_Month', 'First_Day_Month'])
df_forecast = df_forecast[['EAN_TEXT', 'Date', 'Year', 'Month', 'Daily_Forecast']]

In [141]:
df_forecast_90D = df_forecast[(df_forecast.Date >= dt.datetime.today()) & (df_forecast.Date <= dt.datetime.today()+dt.timedelta(days=90))].groupby('EAN_TEXT').sum()['Daily_Forecast'].reset_index()
df_forecast_365D = df_forecast[(df_forecast.Date >= dt.datetime.today()) & (df_forecast.Date <= dt.datetime.today()+dt.timedelta(days=365))].groupby('EAN_TEXT').sum()['Daily_Forecast'].reset_index()

In [142]:
base = df_forecast.copy()

# Inventory

In [143]:
inv = pd.read_csv('../consol_inventario_co/inventory.csv')

In [144]:
inv_to_dist = inv[inv['DHL Pending Maquila'] > 0]

In [145]:
mm_df = pd.read_excel('maquila_master_co.xlsx')
mm_df = mm_df.merge(df_forecast_90D, how='left', left_on='EAN Sales', right_on='EAN_TEXT').drop(columns='EAN_TEXT')
mm_df['Daily_Forecast'] = mm_df['Daily_Forecast'].fillna(0)
mm_df = mm_df.merge(inv[['EAN_TEXT', 'Meli_Full_Available', 'Linio_Available', 'Homecenter Available', 'DHL Available']], how='left', left_on='EAN Sales', right_on='EAN_TEXT').drop(columns='EAN_TEXT')
mm_df['Inv Total'] = mm_df[['Meli_Full_Available', 'Linio_Available', 'Homecenter Available', 'DHL Available']].fillna(0).sum(axis=1)
mm_df.drop(columns=['Meli_Full_Available', 'Linio_Available', 'Homecenter Available', 'DHL Available'], inplace=True)
def required(row, safe=0.1):
    return np.maximum(row['Daily_Forecast'] - row['Inv Total'], 0) + safe*row['Daily_Forecast']
mm_df['required'] = mm_df.apply(lambda row: required(row), axis=1)

In [146]:
primary_components_availability = mm_df.merge(inv[['EAN_TEXT', 'DHL Pending Maquila']], how='left', left_on='EAN Sales', right_on='EAN_TEXT')
primary_components_availability['DHL Pending Maquila'] = primary_components_availability['DHL Pending Maquila'].fillna(0)

In [147]:
primary_components_availability = primary_components_availability[(primary_components_availability['DHL Pending Maquila']> 0)].reset_index(drop=True)

In [148]:
components = pd.DataFrame()
for i in range(len(primary_components_availability)):
    df_ind = pd.DataFrame(json.loads(primary_components_availability.iloc[i]['Components Dict']), index=['Qty']).T.reset_index().rename(columns={'index':'EAN_TEXT'})
    df_ind['Available_Dist'] = df_ind.Qty*primary_components_availability.iloc[i]['DHL Pending Maquila']
    df_ind['Family'] = primary_components_availability.iloc[i]['Family']
    components = pd.concat([components, df_ind])

In [149]:
components = components[['EAN_TEXT', 'Available_Dist', 'Family']].groupby(['EAN_TEXT', 'Family']).sum().reset_index()

In [151]:
mm_df[mm_df.Family == 'Negro Lineas Horizontales']

,Use case,EAN Sales,Family,Brand,Description,Type,Components,Components Dict,Daily_Forecast,Inv Total,required
75,Maletas,EAN07500764002966,Negro Lineas Horizontales,Travel Elite,Juego Set 4 Maletas Rigidas Viaje 4 Ruedas Pol...,Set,"[""EAN07502308501263"",""EAN07502308501300"",""EAN0...","{""EAN07502308501263"":1,""EAN07502308501300"":1,""...",0.000000,466.0,0.000000
76,Maletas,EAN07502308502536,Negro Lineas Horizontales,Travel Elite,"Set 3 Maletas Tour Grande, Mediana, Pequeña 28...",Set,"[""EAN07502308501263"",""EAN07502308501300"",""EAN0...","{""EAN07502308501263"":1,""EAN07502308501300"":1,""...",0.000000,0.0,0.000000
77,Maletas,Pending043,Negro Lineas Horizontales,Travel Elite,"Set 3 Maletas Tour Grande, Mediana, Mano 28, 2...",Set,"[""EAN07502308501263"",""EAN07502308501300"",""EAN0...","{""EAN07502308501263"":1,""EAN07502308501300"":1,""...",0.000000,0.0,0.000000
78,Maletas,Pending044,Negro Lineas Horizontales,Travel Elite,"Set 3 Maletas Tour Grande, Pequeña, Mano 28, 2...",Set,"[""EAN07502308501263"",""EAN07502308501348"",""EAN0...","{""EAN07502308501263"":1,""EAN07502308501348"":1,""...",0.000000,0.0,0.000000
79,Maletas,Pending045,Negro Lineas Horizontales,Travel Elite,"Set 3 Maletas Tour Mediana, Pequeña, Mano 24, ...",Set,"[""EAN07502308501300"",""EAN07502308501348"",""EAN0...","{""EAN07502308501300"":1,""EAN07502308501348"":1,""...",0.000000,0.0,0.000000
80,Maletas,EAN07502308501188,Negro Lineas Horizontales,Travel Elite,Set 2 Maletas Tour Grande y Mano 28 y 20 Pulga...,Set,"[""EAN07502308501263"",""EAN07502308501348""]","{""EAN07502308501263"":1,""EAN07502308501348"":1}",50.021505,391.0,5.002151
81,Maletas,EAN07502308501225,Negro Lineas Horizontales,Travel Elite,Set 2 Maletas Tour Mediana Mano 24 y 16 Pulgad...,Set,"[""EAN07502308501300"",""EAN00733810238652""]","{""EAN07502308501300"":1,""EAN00733810238652"":1}",62.849462,271.0,6.284946
82,Maletas,Pending046,Negro Lineas Horizontales,Travel Elite,Set 2 Maletas Tour Grande y Mediana 28 y 24 Pu...,Set,"[""EAN07502308501263"",""EAN07502308501300""]","{""EAN07502308501263"":1,""EAN07502308501300"":1}",0.000000,0.0,0.000000
83,Maletas,Pending047,Negro Lineas Horizontales,Travel Elite,Set 2 Maletas Tour Grande y Mano 28 y 16 Pulga...,Set,"[""EAN07502308501263"",""EAN00733810238652""]","{""EAN07502308501263"":1,""EAN00733810238652"":1}",0.000000,0.0,0.000000
84,Maletas,Pending048,Negro Lineas Horizontales,Travel Elite,Set 2 Maletas Tour Mediana Mano 24 y 20 Pulgad...,Set,"[""EAN07502308501300"",""EAN07502308501348""]","{""EAN07502308501300"":1,""EAN07502308501348"":1}",0.000000,0.0,0.000000


In [117]:
list(components.Family.unique())

['Gris Lineas Horizontales',
 'Azul Recto',
 'Negro Lineas Horizontales',
 'Tulip Blanco',
 'Tulip Gris',
 'Negro Cruzado Nuevo',
 'Azul Cruzado Nuevo',
 'Plateado Cruzado Nuevo',
 'Rojo Cruzado Nuevo',
 'Tulip Negro',
 'Basica Gris',
 'Basica Blanco',
 'Basica Roja']

In [152]:
#Vars
for family in list(components.Family.unique()):
#for family in ['Negro Lineas Horizontales']:
    m = Model(name='maquila allocation')
    sub_df = mm_df[(mm_df.Family == family) & (mm_df.required > 0)]
    total_required = sub_df.required.sum()
    all_family_eans = list(mm_df[(mm_df.Family == family) & (mm_df.required > 0)]['EAN Sales'])
    comp = list(components[components.Family == family]['EAN_TEXT'])
    eans = []
    if total_required > 0:
        for item in all_family_eans:
            item_components = list(json.loads(list(mm_df[mm_df['EAN Sales'] == item]['Components Dict'].values)[0]))
            #print(item_components)
            if all(item in comp for item in item_components):
                eans.append(item)

        model_variables = eans
        req_variables = []
        avail_variables = []

        #Model Variables
        for ean in eans:
            ean = m.integer_var(name=f'{ean}')

        #Requirement Variables (Forecast - Current Iventory)
        for ean in eans:
            name1 = f'req_{ean}'
            value1 = mm_df[mm_df['EAN Sales'] == ean]['required'].max()
            exec(f'{name1} = {value1}')
            req_variables.append(name1)
            #print(f'{name1} = {globals()[name1]}')

        #Availability Variables (current individual components availability)
        for c in comp:
            name2 = f'avail_{c}'
            value2 = components[components.EAN_TEXT == c]['Available_Dist'].max()
            exec(f'{name2} = {value2}')
            avail_variables.append(name2)
            #print(f'{name2} = {globals()[name2]}')

        #Restrictions
        def cons(ean):
            primary = m.get_var_by_name(ean)
            sets = sub_df[(sub_df['EAN Sales'].isin(eans)) & (sub_df['Components Dict'].str.contains(ean)) & (sub_df['Type'] != 'Ind')].reset_index()
            eans_sales = list(sets['EAN Sales'])
            if primary:
                final = primary
            else:
                final = 0
            for i in range(len(eans_sales)):
                components_dict = json.loads(sets.iloc[i]['Components Dict'])
                comp_qty = components_dict[ean]
                final+=m.get_var_by_name(eans_sales[i])*comp_qty
            return final

        def min_avail(ean):
            primary = m.get_var_by_name(ean)
            sets = sub_df[(sub_df['EAN Sales'].isin(eans)) & (sub_df['Components Dict'].str.contains(ean)) & (sub_df['Type'] != 'Ind')].reset_index()
            #print(list(sets['EAN Sales']))
            prim_comp = []
            for i in range(len(sets)):
                b = list(json.loads(sets.iloc[i]['Components Dict']).keys())
                for ii in range(len(b)):
                    prim_comp.append(b[ii])
            prim_comp = list(set(prim_comp))
            vals = []
            if len(prim_comp) >0:
                for iii in prim_comp:
                    vals.append(globals()[f'avail_{iii}'])
                return vals
            else:
                pass

        for c in comp:
            m.add_constraint(cons(c) == globals()[f'avail_{c}'])
            if min_avail(c):
                m.add_constraint(cons(c) >= m.min(min_avail(c)))

        #Optimization Equation
        def min_array(list):
            min_array = []
            for ean in list:
                value = m.get_var_by_name(ean)/globals()[f'req_{ean}']
                min_array.append(value)
            return min_array

        m.maximize(m.min(min_array(eans)))

        #Model Solver + Concat to Inv DataFrame
        sol = m.solve()
        solutions = sol.__as_df__()
        solutions = solutions[solutions.name.str.contains('EAN')]
        solutions.rename(columns={'name': 'EAN_TEXT', 'value': 'Virtual Inventory - Maquila'}, inplace=True)
        inv = pd.concat([inv, solutions])


In [153]:
sol.__as_df__()

,name,value
0,EAN07752542831804,232.000000
1,EAN00733810238751,83.000000
2,EAN00733810237969,1112.000000
3,EAN07506134951858,60.000000
4,x5,2.726101
5,x6,2.706427
6,x7,2.699248
7,x8,2.736770
8,_min8,2.699248
9,x12,1.000000


In [154]:
m.export_as_lp()

'/tmp/maquila_allocation.lp'

In [134]:
m.__as_df__()

AttributeError: 'Model' object has no attribute '__as_df__'

In [158]:
inv[inv.EAN_TEXT == 'EAN07502308501188']

,EAN_TEXT,Item Name,Brand,Variant,Meli_Full_Available,Meli_Full_Unavailable,Meli_Full_Total,Linio_Available,Homecenter Available,DHL Available,DHL Pending Maquila,DHL Raw Materials,DHL Spare Parts,Virtual Inventory - Maquila,DHL Available Post Maquila
145,EAN07502308501188,Set Kit 2 Maletas Rigidas Grande Bodega Mano 4...,Travel Elite,Negro Lineas Horizontales,41.0,0.0,41.0,9.0,0.0,341.0,0.0,0.0,0.0,1242.0,1583.0


In [156]:
ean_details = inv[inv.Brand.isnull() == False][['EAN_TEXT', 'Item Name', 'Brand']]

In [157]:
inv.drop(columns=['Brand', 'Item Name'], inplace=True)
inv = inv.reset_index(drop=True)
inv = inv.fillna(0)
inv = inv.groupby('EAN_TEXT').sum().reset_index()
inv = inv.merge(mm_df[['EAN Sales', 'Family', 'Daily_Forecast', 'required', 'Description', 'Brand']], how='left', left_on='EAN_TEXT', right_on='EAN Sales')
inv = inv.merge(ean_details, how='left', on='EAN_TEXT')
inv['Brand_y'] = inv['Brand_y'].fillna(inv['Brand_x'])
inv['Item Name'] = inv['Item Name'].fillna(inv['Description'])
inv.drop(columns=['Brand_x', 'Description', 'EAN Sales'], inplace=True)
inv.rename(columns={'Brand_y':'Brand', 'Family':'Variant', 'Daily_Forecast':'Forecast Period'}, inplace=True)
inv['DHL Available Post Maquila'] = inv['DHL Available'] + inv['Virtual Inventory - Maquila']
inv = inv[['EAN_TEXT', 
           'Item Name', 
           'Brand', 
           'Variant', 
           'Meli_Full_Available', 
           'Meli_Full_Unavailable',
           'Meli_Full_Total',
           'Linio_Available',
           'Homecenter Available',
           'DHL Available',
           'DHL Pending Maquila',
           'DHL Raw Materials',
           'DHL Spare Parts', 
           'Virtual Inventory - Maquila', 
           'DHL Available Post Maquila',
           #'Forecast Period', 
           #'required'
          ]]

In [159]:
inv['Date'] = pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d'))

In [160]:
inventory_for_buyplan = inv.copy()
inventory_for_buyplan['Total Available']  = inventory_for_buyplan['DHL Available Post Maquila'] + inventory_for_buyplan['Meli_Full_Available'] + inventory_for_buyplan['Linio_Available'] + inventory_for_buyplan['Homecenter Available']

In [161]:
base = base.merge(inventory_for_buyplan[['EAN_TEXT', 'Variant', 'Meli_Full_Available',
       'Meli_Full_Unavailable', 'Meli_Full_Total', 'Linio_Available',
       'Homecenter Available', 'DHL Available', 'DHL Pending Maquila',
       'DHL Raw Materials', 'DHL Spare Parts',
       'Virtual Inventory - Maquila', 'DHL Available Post Maquila', 'Date',
       'Total Available']], how='left', left_on=['EAN_TEXT', 'Date'], right_on=['EAN_TEXT', 'Date'])

In [162]:
base = base.sort_values(by=['EAN_TEXT', 'Date'], ascending=[True, True]).reset_index().drop(columns=['index'])

In [163]:
base['Total Available'] = base['Total Available'].fillna(0)

# POs

In [164]:
pos = pd.read_csv('e2e_sourcing.csv', encoding = "ISO-8859-1")
pos = pos[(pos['ships_to.country'] == 'CO') & ((pos['status'] == 'active') | (pos['status'] == 'shipped'))]

In [165]:
pos['sku'] = pos['sku'].fillna('').astype(str)
pos['sku'] = pos['sku'].str.strip()
pos['EAN_TEXT'] = "EAN"+("000000000"+pos['sku']).str[-14:]
pos = pos.dropna(subset='best_eta')
pos['best_eta'] = pd.to_datetime(pos['best_eta'])
pos = pos.reset_index().drop(columns='index')
pos['name_y'] = pos['name_y'].str.replace('\n',' ', regex=False)

In [166]:
mm2_df = pd.read_excel('maquila_master_co.xlsx')
mm2_df = mm2_df.merge(df_forecast_365D, how='left', left_on='EAN Sales', right_on='EAN_TEXT').drop(columns='EAN_TEXT')
mm2_df['Daily_Forecast'] = mm2_df['Daily_Forecast'].fillna(0)
mm2_df = mm2_df.merge(inv[['EAN_TEXT', 'Meli_Full_Available', 'Linio_Available', 'Homecenter Available', 'DHL Available Post Maquila']], how='left', left_on='EAN Sales', right_on='EAN_TEXT').drop(columns='EAN_TEXT')
mm2_df['Inv Total'] = mm2_df[['Meli_Full_Available', 'Linio_Available', 'Homecenter Available', 'DHL Available Post Maquila']].fillna(0).sum(axis=1)
mm2_df.drop(columns=['Meli_Full_Available', 'Linio_Available', 'Homecenter Available', 'DHL Available Post Maquila'], inplace=True)
def required(row, safe=0.1):
    return np.maximum(row['Daily_Forecast'] - row['Inv Total'], 0) + safe*row['Daily_Forecast']
mm2_df['required'] = mm2_df.apply(lambda row: required(row), axis=1)

In [167]:
maquila_eans = list(set(mm2_df['EAN Sales']))
pos_to_distribute = pos[pos.EAN_TEXT.isin(maquila_eans)].reset_index(drop=True)

In [168]:
added_rows = []
for i in range(len(pos_to_distribute)):
    m = Model(name='maquila PO allocation')
    row = pos_to_distribute.iloc[i].to_dict()
    ean_to_distribute = pos_to_distribute.iloc[i]['EAN_TEXT']
    family = mm2_df[mm2_df['EAN Sales'] == ean_to_distribute]['Family'].values[0]
    sub_df = mm2_df[(mm2_df.Family == family) & (mm2_df.required > 0)]
    all_family_eans = list(mm2_df[(mm2_df.Family == family) & (mm2_df.required > 0)]['EAN Sales'])
    ean_to_distribute_components = list(json.loads(list(mm2_df[mm2_df['EAN Sales'] == ean_to_distribute]['Components Dict'].values)[0]))
    available_components_quantity = pos_to_distribute.iloc[i]['quantity']
    eans = []
    for item in all_family_eans:
        item_components = list(json.loads(list(mm2_df[mm2_df['EAN Sales'] == item]['Components Dict'].values)[0]))
        if all(item in ean_to_distribute_components for item in item_components):
            eans.append(item)

     #Model Variables
    for ean in eans:
        ean = m.integer_var(name=f'{ean}')

    #Requirement Variables (Forecast - Current Iventory)
    for ean in eans:
        name1 = f'req_{ean}'
        value1 = mm2_df[mm2_df['EAN Sales'] == ean]['required'].max()
        exec(f'{name1} = {value1}')
        req_variables.append(name1)

    #Availability Variables (current individual components availability)
    for c in ean_to_distribute_components:
        name2 = f'avail_{c}'
        value2 = available_components_quantity
        exec(f'{name2} = {value2}')
        avail_variables.append(name2)
        #print(f'{name2} = {globals()[name2]}')

    #Restrictions
    def cons(ean):
        primary = m.get_var_by_name(ean)
        sets = sub_df[(sub_df['EAN Sales'].isin(eans)) & (sub_df['Components Dict'].str.contains(ean)) & (sub_df['Type'] != 'Ind')].reset_index()
        eans_sales = list(sets['EAN Sales'])
        if primary:
            final = primary
        else:
            final = 0
        for i in range(len(eans_sales)):
            components_dict = json.loads(sets.iloc[i]['Components Dict'])
            comp_qty = components_dict[ean]
            final+=m.get_var_by_name(eans_sales[i])*comp_qty
        return final

    def min_avail(ean):
        primary = m.get_var_by_name(ean)
        sets = sub_df[(sub_df['EAN Sales'].isin(eans)) & (sub_df['Components Dict'].str.contains(ean)) & (sub_df['Type'] != 'Ind')].reset_index()
        #print(list(sets['EAN Sales']))
        prim_comp = []
        for i in range(len(sets)):
            b = list(json.loads(sets.iloc[i]['Components Dict']).keys())
            for ii in range(len(b)):
                prim_comp.append(b[ii])
        prim_comp = list(set(prim_comp))
        vals = []
        if len(prim_comp) >0:
            for iii in prim_comp:
                vals.append(globals()[f'avail_{iii}'])
            return vals
        else:
            pass

    for c in ean_to_distribute_components:
        m.add_constraint(cons(c) == globals()[f'avail_{c}'])
        if min_avail(c):
            m.add_constraint(cons(c) >= m.min(min_avail(c)))

    #Optimization Equation
    def min_array(list):
        min_array = []
        for ean in list:
            value = m.get_var_by_name(ean)/globals()[f'req_{ean}']
            min_array.append(value)
        return min_array

    m.maximize(m.min(min_array(eans)))

    sol = m.solve()
    solutions = sol.__as_df__()
    solutions = solutions[solutions.name.str.contains('EAN')]
    m.end()

    for i_sol in range(len(solutions)):
        total_available = len(ean_to_distribute_components)*available_components_quantity
        ean_to_distribute_components_count = len(ean_to_distribute_components)
        number_of_components = len(list(json.loads(list(mm2_df[mm2_df['EAN Sales'] == solutions.iloc[i_sol]['name']]['Components Dict'].values)[0])))
        desc = mm2_df[mm2_df['EAN Sales'] == solutions.iloc[i_sol]['name']]['Description'].values[0]
        new_row = row.copy()
        new_row['name_y'] = desc
        new_row['sku'] = solutions.iloc[i_sol]['name'][-13:]
        new_row['EAN_TEXT'] = solutions.iloc[i_sol]['name']
        new_row['quantity'] = solutions.iloc[i_sol]['value']
        new_row['total_cost'] = new_row['unit_price']/ean_to_distribute_components_count*solutions.iloc[i_sol]['value']*number_of_components
        new_row['unit_price'] = new_row['total_cost']/new_row['quantity']
        added_rows.append(new_row)


In [169]:
maquila_pos = pd.DataFrame(added_rows)
maquila_pos['maquila'] = 'yes'

In [170]:
nonmaquila_pos = pos[pos.EAN_TEXT.isin(maquila_eans) == False]
nonmaquila_pos['maquila'] = 'no'

/tmp/ipykernel_15658/3895101850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonmaquila_pos['maquila'] = 'no'


In [171]:
pos_dist = pd.concat([maquila_pos, nonmaquila_pos]).reset_index(drop=True)

In [172]:
current_day = datetime.today().astimezone(local_tz).strftime('%Y%m%d')
pos_dist.to_csv(f'{current_day}_pos_dist.csv')

In [173]:
pos_dist_by_date = pos_dist[['EAN_TEXT', 'best_eta', 'quantity', 'maquila']].groupby(['EAN_TEXT', 'best_eta', 'maquila']).sum().reset_index()

In [174]:
base = base.merge(pos_dist_by_date, how='left', left_on=['EAN_TEXT', 'Date'], right_on=['EAN_TEXT', 'best_eta'])

In [175]:
base.columns

Index(['EAN_TEXT', 'Date', 'Year', 'Month', 'Daily_Forecast', 'Variant',
       'Meli_Full_Available', 'Meli_Full_Unavailable', 'Meli_Full_Total',
       'Linio_Available', 'Homecenter Available', 'DHL Available',
       'DHL Pending Maquila', 'DHL Raw Materials', 'DHL Spare Parts',
       'Virtual Inventory - Maquila', 'DHL Available Post Maquila',
       'Total Available', 'best_eta', 'maquila', 'quantity'],
      dtype='object')

In [176]:
base = base[['EAN_TEXT', 'Date', 'Year',
       'Month', 'Daily_Forecast', 'Total Available', 'quantity', 'maquila']]
base = base.rename(columns={'quantity':'Scheduled Arrivals'})

In [177]:
base['Scheduled Arrivals'] = base['Scheduled Arrivals'].fillna(0)
base['Daily_Forecast'] = base['Daily_Forecast'].fillna(0)

In [178]:
eans = list(base.EAN_TEXT.unique())


In [179]:
base2 = pd.DataFrame()

for ean in eans:
    ind_base = base[base.EAN_TEXT == ean].reset_index(drop=True)
    for i in range(len(ind_base)):
        if ind_base.iloc[i]['Date'] == pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d')):                  
            ind_base.at[i, 'Total Available'] = np.maximum(0,ind_base.iloc[i]['Total Available'] - ind_base.iloc[i]['Daily_Forecast'] + ind_base.iloc[i]['Scheduled Arrivals'])
            ind_base.at[i, 'Total Available Sales'] = np.maximum(0,ind_base.iloc[i]['Total Available'] - ind_base.iloc[i]['Daily_Forecast'])
        else:
            ind_base.at[i, 'Total Available'] = np.maximum(0,ind_base.iloc[i-1]['Total Available'] - ind_base.iloc[i]['Daily_Forecast'] + ind_base.iloc[i]['Scheduled Arrivals'])
            if ind_base.iloc[i]['maquila'] == 'yes':
                ind_base.at[i, 'Total Available Sales'] = np.maximum(0,ind_base.iloc[i-1]['Total Available Sales'] - ind_base.iloc[i]['Daily_Forecast'] + ind_base.iloc[i-15]['Scheduled Arrivals'])
            else:
                ind_base.at[i, 'Total Available Sales'] = np.maximum(0,ind_base.iloc[i-1]['Total Available Sales'] - ind_base.iloc[i]['Daily_Forecast'] + ind_base.iloc[i-7]['Scheduled Arrivals'])
    base2 = pd.concat([base2, ind_base])

In [180]:
base2 = base2.reset_index(drop=True)

In [181]:
base = base2

# Next OOS

In [182]:
next_oos_date = base[base['Total Available'] > 0]

In [183]:
next_oos_date = next_oos_date[['EAN_TEXT', 'Date']].groupby(['EAN_TEXT']).max()

In [184]:
next_oos_date = next_oos_date.reset_index()

# Next OOS - Immediate Next OOS

In [185]:
next_oos_cold = base[base['Total Available'] == 0]

In [186]:
next_oos_cold = next_oos_cold[['EAN_TEXT', 'Date']].groupby(['EAN_TEXT']).min()

In [187]:
next_oos_cold = next_oos_cold.reset_index()

# Leadtime and Priority

In [188]:
lt_prior = pd.read_csv('Leadtime Priority Master CO.csv')

In [189]:
base = base.merge(lt_prior[['EAN_TEXT', 'Source', 'Leadtime', 'Leadtime ex sourcing', 'Priority', 'Descripcion', 'Brand', 'Status']], how='left', left_on='EAN_TEXT', right_on='EAN_TEXT')

In [190]:
base['Leadtime'] = base['Leadtime'].fillna(base[base.Priority.isnull() == False]['Leadtime'].mean())

In [191]:
base[base['Priority'].isnull()].to_csv('inlcude_in_priority.csv')

In [192]:
base['Priority'] = base['Priority'].fillna('P1')

# Buyplan

In [193]:
buyplan = base.copy()
buyplan = buyplan.dropna(subset='Source')
buyplan = buyplan[(buyplan.Date <= '2024-12-31')]

In [194]:
## Inputs
P0_safety = 15
P1_safety = 10
P2_safety = 5
EANS = buyplan.EAN_TEXT.unique()
P0_policy = 45
P1_policy = 30
P2_policy = 20

In [195]:
final_bp = []
for ean in EANS:
    ind_bp = buyplan[buyplan.EAN_TEXT == ean].reset_index(drop=True)
    ind_bp['PO Volume'] = 0
    ind_bp['Theoretical Arrivals'] = 0
    if len(next_oos_date[next_oos_date.EAN_TEXT == ean]['Date'].reset_index(drop=True)) >0:
        oos_date = next_oos_date[next_oos_date.EAN_TEXT == ean]['Date'].reset_index(drop=True)
    else:
        oos_date = pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d'))
    priority = ind_bp.iloc[0]['Priority']
    if ind_bp.iloc[0]['Source'] == 'Import':
        leadtime = ind_bp['Leadtime'].max() + 30
    else:
        leadtime = ind_bp['Leadtime'].max()
    #leadtime = ind_bp['Leadtime'].max()
    if priority == 'P0':
        safety = P0_safety
        policy = P0_policy
    elif priority == 'P1':
        safety = P1_safety
        policy = P1_policy
    else:
        safety = P2_safety
        policy = P2_policy
    try:
        po_date = oos_date.at[0] - timedelta(days=(safety+leadtime))
    except:
        po_date = oos_date - timedelta(days=(safety+leadtime))
    if po_date < pd.Timestamp(datetime.today().astimezone(local_tz).strftime('%Y%m%d')):
        po_date = pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d'))
    while po_date < ind_bp.Date.max():
        if po_date == pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d')):
            po_volume = ind_bp[(ind_bp.Date > po_date + timedelta(days=leadtime)) & (ind_bp.Date <= po_date + timedelta(days=safety+leadtime+policy))]['Daily_Forecast'].sum()
        else:
            po_volume = ind_bp[(ind_bp.Date > po_date + timedelta(days=safety+leadtime)) & (ind_bp.Date <= po_date + timedelta(days=safety+leadtime+policy))]['Daily_Forecast'].sum()
        index = ind_bp.index[ind_bp.Date == po_date].to_list()[0]
        ind_bp.at[index, 'PO Volume'] = po_volume
        try:
            ind_bp.at[index+leadtime, 'Theoretical Arrivals'] = po_volume
        except:
            pass
        po_date = po_date + timedelta(days=policy)
    final_bp.append(ind_bp)
final_bp = pd.concat(final_bp)

In [196]:
final_bp = final_bp.reset_index(drop=True)

In [197]:
final_bp = final_bp.dropna(subset='EAN_TEXT')

# All Together

In [198]:
base = base.merge(final_bp[['EAN_TEXT', 'Date', 'PO Volume', 'Theoretical Arrivals']], how='left', left_on=['EAN_TEXT', 'Date'], right_on=['EAN_TEXT', 'Date'])

In [199]:
base['PO Volume'] = base['PO Volume'].fillna(0)
base['Theoretical Arrivals'] = base['Theoretical Arrivals'].fillna(0)

In [200]:
for i in range(len(base)):
    if base.iloc[i]['Date'] == pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d')):                  
        base.at[i, 'Modeled Available'] = np.maximum(0,base.iloc[i]['Total Available'] + base.iloc[i]['Theoretical Arrivals'])
    else:
        base.at[i, 'Modeled Available'] = np.maximum(0,base.iloc[i-1]['Modeled Available'] - base.iloc[i]['Daily_Forecast'] + base.iloc[i]['Scheduled Arrivals'] + base.iloc[i]['Theoretical Arrivals'])

In [201]:
base['Forecasted Sales'] = np.minimum(base['Total Available Sales'], base['Daily_Forecast'])

In [202]:
base['Descripcion'] = base['Descripcion'].str.replace('\n',' ', regex=False)

In [203]:
buyplan_co = base[['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Year',
       'Month', 'Leadtime', 'Leadtime ex sourcing', 'Priority', 'PO Volume']].groupby(['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Year',
       'Month', 'Leadtime', 'Leadtime ex sourcing', 'Priority']).sum().reset_index()

In [204]:
buyplan_co = buyplan_co.pivot(index=['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Leadtime', 'Leadtime ex sourcing', 'Priority'], columns=['Year', 'Month'], values='PO Volume').reset_index()

In [205]:
buyplan_co.columns = buyplan_co.columns.to_flat_index()

In [206]:
cols = []
for column in buyplan_co.columns:
    cols.append(' '.join(map(str,column)).strip())
buyplan_co.columns = cols

In [207]:
buyplan_co = buyplan_co.rename(columns={'Status': 'Active'})

In [208]:
buyplan_co = buyplan_co.merge(next_oos_date, how='left', on='EAN_TEXT')

In [209]:
buyplan_co.rename(columns={'Date':'Predicted OOS Date'}, inplace=True)

In [210]:
buyplan_co = buyplan_co.merge(next_oos_cold, how='left', on='EAN_TEXT')

In [211]:
buyplan_co.rename(columns={'Date':'Predicted Immediate OOS Date'}, inplace=True)

In [212]:
def required_ETA(row):
    try:
        if row['Priority'] == 'P0':
            return row['Predicted OOS Date'] - timedelta(days=P0_safety)
        elif row['Priority'] == 'P1':
            return row['Predicted OOS Date'] - timedelta(days=P1_safety)
        else:
            return row['Predicted OOS Date'] - timedelta(days=P2_safety)
    except:
        return None

In [213]:
buyplan_co['Required_ETA'] = buyplan_co.apply(lambda row: required_ETA(row), axis=1)

In [214]:
buyplan_co['Predicted OOS Date'] = buyplan_co['Predicted OOS Date'].fillna(pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d')))

In [215]:
buyplan_co['Predicted Immediate OOS Date'] = buyplan_co['Predicted Immediate OOS Date'].fillna(pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d')))

In [216]:
buyplan_co['Required_ETA'] = buyplan_co['Required_ETA'].fillna(pd.to_datetime(datetime.today().astimezone(local_tz).strftime('%Y%m%d')))

In [217]:
buyplan_co.to_csv(f'{current_day}_buyplan_co.csv', index=False)

In [218]:
forecast1 = pd.read_csv('Forecast.csv')

In [219]:
cols_to_keep = ['EAN_TEXT', 'Forecast 2023 3', 'Forecast 2023 4',
       'Forecast 2023 5', 'Forecast 2023 6', 'Forecast 2023 7',
       'Forecast 2023 8', 'Forecast 2023 9', 'Forecast 2023 10',
       'Forecast 2023 11', 'Forecast 2023 12', 'Forecast 2024 1',
       'Forecast 2024 2']

In [220]:
forecast1 = forecast1[cols_to_keep]

In [221]:
buyplan_co_w_forecast = buyplan_co.merge(forecast1, how='left', left_on='EAN_TEXT', right_on='EAN_TEXT')

In [222]:
buyplan_co_w_forecast = buyplan_co_w_forecast.merge(inventory_for_buyplan[['EAN_TEXT', 'Total Available']], how='left', on='EAN_TEXT')

In [223]:
buyplan_co_w_forecast = buyplan_co_w_forecast.rename(columns={'Total Available': 'Total Available Inventory'})

In [224]:
pos_dist_by_month = pos_dist_by_date.copy()

In [225]:
pos_dist_by_month = pos_dist_by_month[pos_dist_by_month.best_eta > datetime.today() + timedelta(days=-7)]

In [226]:
pos_dist_by_month = pos_dist_by_month.rename(columns={'quantity': 'Expected Arrivals'})

In [227]:
pos_dist_by_month['Month'] = pd.DatetimeIndex(pos_dist_by_month['best_eta']).month
pos_dist_by_month['Year'] = pd.DatetimeIndex(pos_dist_by_month['best_eta']).year

In [228]:
pos_dist_by_month = pos_dist_by_month.sort_values(by='best_eta', axis=0, ascending=True)

In [229]:
pos_dist_by_month = pos_dist_by_month[['EAN_TEXT', 'Month', 'Year', 'Expected Arrivals']].groupby(['EAN_TEXT', 'Month', 'Year']).sum().reset_index()

In [230]:
pos_dist_by_month = pos_dist_by_month.pivot(index=['EAN_TEXT'], columns=['Year', 'Month'], values=['Expected Arrivals']).reset_index()

In [231]:
cols = []
for column in pos_dist_by_month.columns:
    cols.append(' '.join(map(str,column)).strip())
pos_dist_by_month.columns = cols

In [232]:
cols.sort()

In [233]:
pos_dist_by_month = pos_dist_by_month[cols]

In [234]:
pos_dist_by_month = pos_dist_by_month.fillna(0)

In [235]:
buyplan_co_w_forecast = buyplan_co_w_forecast.merge(pos_dist_by_month, how='left', on='EAN_TEXT')

In [236]:
buyplan_co_w_forecast.to_csv('Buyplan_Razon_Validation_Colombia.csv', index=False)

In [237]:
base.to_csv('base.csv', index=False)

In [238]:
buyplan_co_dash = pd.melt(buyplan_co, id_vars=['EAN_TEXT', 'Brand', 'Descripcion', 'Active', 'Leadtime','Leadtime ex sourcing', 'Priority', 'Predicted OOS Date', 'Predicted Immediate OOS Date', 'Required_ETA'], value_vars=list(buyplan_co.iloc[:,7:-3].columns))
buyplan_co_dash['value'] = buyplan_co_dash['value'].apply(lambda x: math.ceil(x))
buyplan_co_dash['Year'] = buyplan_co_dash['variable'].apply(lambda x: x.split(' ')[0])
buyplan_co_dash['Month'] = buyplan_co_dash['variable'].apply(lambda x: x.split(' ')[1])
buyplan_co_dash = buyplan_co_dash.drop(columns=['variable'])
buyplan_co_dash.rename(columns={'value': 'PO Units'}, inplace=True)

In [239]:
buyplan_co_dash.to_csv('buyplan_co_dash.csv')
base[base.Year <= 2023].to_csv('buyplan_co_base_for_dash.csv')

# Scheduled Arrivals

In [101]:
scheduled_arrivals = base.copy()

In [102]:
scheduled_arrivals = scheduled_arrivals[['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Year',
       'Month', 'Leadtime', 'Leadtime ex sourcing', 'Priority', 'Scheduled Arrivals','Daily_Forecast', 'Forecasted Sales']].groupby(['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Year',
       'Month', 'Leadtime', 'Leadtime ex sourcing', 'Priority']).sum().reset_index()

In [103]:
scheduled_arrivals = scheduled_arrivals.rename(columns={'Daily_Forecast': 'Forecast'})

In [104]:
scheduled_arrivals = scheduled_arrivals.pivot(index=['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Leadtime', 'Leadtime ex sourcing', 'Priority'], columns=['Year', 'Month'], values=['Forecast', 'Forecasted Sales', 'Scheduled Arrivals']).reset_index()

In [105]:
scheduled_arrivals.columns = scheduled_arrivals.columns.to_flat_index()

In [106]:
cols = []
for column in scheduled_arrivals.columns:
    cols.append(' '.join(map(str,column)).strip())
scheduled_arrivals.columns = cols

In [107]:
scheduled_arrivals.to_csv(f'{current_day}_noche_file.csv', index=False)

# Constrained Forecast

In [108]:
constrained_forecast = base[base.Date <= '2023-12-31']

In [109]:
constrained_forecast = constrained_forecast[['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Year',
       'Month', 'Leadtime', 'Leadtime ex sourcing', 'Priority', 'Daily_Forecast', 'Forecasted Sales']].groupby(['EAN_TEXT', 'Brand', 'Descripcion', 'Status', 'Year',
       'Month', 'Leadtime', 'Leadtime ex sourcing', 'Priority']).sum().reset_index()

In [110]:
constrained_forecast = constrained_forecast.rename(columns={'Daily_Forecast': 'Forecast'})

In [111]:
constrained_forecast = constrained_forecast.pivot(index=['EAN_TEXT','Brand', 'Descripcion', 'Status', 'Leadtime', 'Leadtime ex sourcing', 'Priority'], columns=['Year', 'Month'], values=['Forecast', 'Forecasted Sales']).reset_index()

In [112]:
constrained_forecast.columns = constrained_forecast.columns.to_flat_index()

In [113]:
cols = []
for column in constrained_forecast.columns:
    cols.append(' '.join(map(str,column)).strip())
constrained_forecast.columns = cols

In [114]:
constrained_forecast.to_csv(f'{current_day}_constrained_sales.csv', index=False)